<a href="https://colab.research.google.com/github/Fa1c0n35/-pantagrule/blob/master/Image_Classification_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

##Pipeline
1. get data
2. extract feature
3. train model
4. test model
5. deploy model

In [ ]:
!pip uninstall keras
!pip uninstall tensorflow
!pip install keras==2.4.3
!pip install tensorflow==2.4.3
!pip install botnoi==0.2.1

Found existing installation: keras 2.6.0
Uninstalling keras-2.6.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? ัy
Your response ('ัy') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled keras-2.6.0
Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
y
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 394.5 MB 338 bytes/s 
     |█████████████████████████

In [ ]:
import botnoi as bn
from botnoi import scrape as sc
from botnoi import cv
import os

In [ ]:
def extractimagefeat(query):
  #create folder
  foldername = 'images/'+query
  isdir = os.path.isdir(foldername) 
  #check if folder exist
  if not isdir:
    #create directory
    os.makedirs(foldername)
  #get images from google search
  imglist = sc.get_image_urls(query,30)
  i = 1
  for img in imglist[0:30]:
    #extract image features from each images and save to files
    try:
      print(i)
      #create image path
      savepath = foldername + '/' + str(i)+'.p'
      a = cv.image(img)
      a.getresnet50()
      #a.getmobilenet()
      #a.getmobilenet()
      a.save(savepath)
      i = i + 1
    except:
      pass
  return 'complete'


In [ ]:
import glob
import pandas as pd
import pickle
def createdataset(clsList):
  #imgfolder = glob.glob('images/*')
  dataset = []
  for cls in clsList:
    clsset = pd.DataFrame()
    pList = glob.glob('images/'+cls+'/*')
    featvec = []
    for p in pList:
      dat = pickle.load(open(p,'rb'))
      featvec.append(dat.resnet50)
    clsset['feature'] = featvec
    cls = cls.split('/')[-1]
    clsset['label'] = cls
    dataset.append(clsset)
  return pd.concat(dataset,axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import LinearSVC
def trainmodel(dataset,modfile=''):
  trainfeat, testfeat, trainlabel, testlabel = train_test_split(dataset['feature'], dataset['label'], test_size=0.20, random_state=0)
  clf = LinearSVC()
  mod = clf.fit(np.vstack(trainfeat.values),trainlabel.values)
  res = mod.predict(np.vstack(testfeat.values))
  if modfile!='':
    pickle.dump(mod,open(modfile,'wb'))
  acc = sum(res == testlabel)/len(res)
  return mod,acc

In [ ]:
def createclassifier(clsList,modfile):
  for c in clsList:
    extractimagefeat(c)
  dset = createdataset(clsList)
  return trainmodel(dset,modfile)

In [ ]:
createclassifier(['ไข่ไก่','ไข่เป็ด'],'ไข่ไก่ไข่เป็ด.mod')

complete: 0.00%
completed
1
2
3
4
5
5
6
7
7
8
9
10
11
12
12
13
13
14
15
16
17
18
18
19
20
21
22
23
24
25
complete: 0.00%
completed
1
2
3
4
4
4
4
5
5
6
7
8
8
9
10
11
12
13
14
15
16
17
18
19
20
20
21
22
23
24


(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
           intercept_scaling=1, loss='squared_hinge', max_iter=1000,
           multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
           verbose=0), 0.6)

In [ ]:
# output function
modFile = 'ไข่ไก่ไข่เป็ด.mod'
mod = pickle.load(open(modFile,'rb'))

In [ ]:
def predicting(imgurl):
  a = cv.image(imgurl)
  feat = a.getresnet50()
  res = mod.predict([feat])
  return res

In [ ]:
a=predicting('https://www.kasethub.co.th/pic/bigpic/869f9476ed39fd80bf1dc7cf628dcd1b.jpg')
# a=predictimg('https://f.ptcdn.info/810/034/000/1440553775-IMG5594-o.jpg')
a

array(['ไข่เป็ด'], dtype=object)

In [ ]:
def getthumbdisplay(imgList):
  res = ''
  for img in imgList:
    res = res + gethtmlimage(img)
  return res


In [ ]:
def gethtmlimage(imgurl):
  htmlTag = '''
  <a target="_blank" href="%s">
    <img src="%s" alt="Forest">
  </a>
  '''%(imgurl,imgurl)
  return htmlTag


In [ ]:
import IPython
from google.colab import output
def displayimagelist(imgList):
  htmlimage = getthumbdisplay(imgList)
  html = '''<style>
  img {
    border: 1px solid #ddd; /* Gray border */
    border-radius: 4px;  /* Rounded border */
    padding: 5px; /* Some padding */
    width: 150px; /* Set a small width */
  }

  /* Add a hover effect (blue shadow) */
  img:hover {
    box-shadow: 0 0 2px 1px rgba(0, 140, 186, 0.5);
  }
  </style>
  <body>

  %s

  </body>'''%htmlimage
  display(IPython.display.HTML(html))

In [ ]:
imglist = sc.get_image_urls('ลิซ่า',30)
displayimagelist(imglist)

complete: 0.00%
completed
